In [1]:
from elasticsearch import Elasticsearch
from os import listdir
from os.path import isfile, join

In [5]:
client = Elasticsearch([
    {'host': 'localhost'},
])

In [6]:
create_index_body = {
      'settings': {
        'number_of_shards': 1,
        'number_of_replicas': 0,

        'analysis': {
          'analyzer': {
            'bill_analyzer': {
              'type': 'custom',
              'tokenizer': 'standard',
              'filter': ['morfologik_stem', 'lowercase', "filter_shingle"]
            }
          },
        "filter": {
         "filter_shingle": {
          "type": "shingle",
          "max_shingle_size": 2,
          "min_shingle_size": 2,
          "output_unigrams": False
        }
      }
        }
      },
      'mappings': {
        'bill': {
          'properties': {
            'content': {'type': 'text', 'analyzer': 'bill_analyzer'},
          }
        }
      }
    }

In [7]:
client.indices.create(    
    index='bill_index2',
     body=create_index_body)

PUT http://localhost:9200/bill_index2 [status:400 request:0.030s]


RequestError: RequestError(400, 'resource_already_exists_exception', 'index [bill_index2/FSlXsBzUQ1WgtWtmGI1KXQ] already exists')

In [49]:
def load_data(directory_path):
    files = [file for file in listdir(directory_path) if isfile(join(directory_path, file))]
    for file in files:
        with open(f"{directory_path}/{file}") as input_file:
            client.index(index='bill_index2', id=f"{file}", doc_type='bill', body={'content':input_file.read() })

In [50]:
load_data('./data/')

In [8]:
def get_ids(directory_path):
    files = [file for file in listdir(directory_path) if isfile(join(directory_path, file))]
    ids = map(lambda file: f"{file}",files)
    return list(ids)

In [9]:
ids = get_ids('./data/')

In [10]:
global_frequency = {}

In [11]:
def get_global_term_frequency(terms_data):
    for term in terms_data:
        if term not in global_frequency:
            global_frequency[term] = terms_data[term]['ttf']

In [12]:
def get_terms_data(term_vectors):
    return term_vectors['term_vectors']['content']['terms']

In [13]:
def get_terms(terms_data):
    return terms_data.keys()

In [14]:
for id_ in ids:
    term_vectors = client.termvectors(index='bill_index2', doc_type='bill', id=id_, body= {
      "fields" : ["content"],
      "offsets" : True,
      "payloads" : True,
      "positions" : True,
      "term_statistics" : True,
      "field_statistics" : True
    })
    term_data = get_terms_data(term_vectors)
    get_global_term_frequency(term_data)

In [4]:
global_frequency = deserialize('bigram_freq.pickle')

In [5]:
filtered_bigrams = {}

In [6]:
for key in global_frequency:
    if len(key.split(' ')) > 2:
        continue
    first, second = key.split(' ')[:2]
    if first.isalpha() and second.isalpha():
        filtered_bigrams[key] = global_frequency[key]

In [7]:
import pickle
with open('global_frequency.pickle', 'rb') as handle:
    unigram_frequency =  pickle.load(handle)

In [8]:
sorted_bigrams = sorted(filtered_bigrams.items(), key=lambda entry: entry[1] , reverse=True)

In [9]:
sorted_bigrams[:30]

[('w wiek', 188282),
 ('o ocean', 65201),
 ('ocean ojciec', 65201),
 ('lub lubić', 46100),
 ('wiek artykuł', 32202),
 ('ojciec który', 28762),
 ('który mowa', 28644),
 ('mowa w', 28576),
 ('wiek usta', 23585),
 ('nie on', 23070),
 ('brzmienie brzmieć', 17975),
 ('rok numer', 17943),
 ('od oda', 16124),
 ('określić w', 13934),
 ('określony określić', 13471),
 ('może móc', 12148),
 ('z dzień', 11416),
 ('ten to', 10642),
 ('otrzymywać brzmienie', 10587),
 ('sprawa sprawić', 10037),
 ('prawo prawy', 9651),
 ('ustawa z', 8811),
 ('albo alba', 8760),
 ('do sprawa', 8737),
 ('i numer', 8634),
 ('właściwy do', 8563),
 ('postępowanie postępować', 8549),
 ('dodawać się', 8235),
 ('minister właściwy', 7958),
 ('wiek brzmienie', 7329)]

In [ ]:
serialize('bigram_freq')

In [13]:
import math
def pmi(bigram):
    try:
        word1, word2 = bigram.split(' ')[:2]
        prob_word1 = unigram_frequency[word1] / float(sum(unigram_frequency.values()))
        prob_word2 = unigram_frequency[word1] / float(sum(unigram_frequency.values()))
        prob_word1_word2 = filtered_bigrams[bigram] / float(sum(filtered_bigrams.values()))

        return math.log(prob_word1_word2/float(prob_word1*prob_word2),2)
    except: # Occurs when calculating PMI for Out-of-Vocab words.
        return 0

In [14]:
pmi_dict = {}

In [15]:
for bigram in filtered_bigrams:
    pmi_dict[bigram] = pmi(bigram)

In [2]:
import pickle
def serialize(name, dictionary):
    with open(name, 'wb') as handle:
        pickle.dump(dictionary, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [3]:
def deserialize(name):
    with open(name, 'rb') as handle:
        return pickle.load(handle)

In [18]:
serialize('pmi_dict.pickle', pmi_dict)

In [19]:
sorted_pmi = sorted(pmi_dict.items(), key=lambda entry: entry[1] , reverse=True)

In [20]:
sorted_pmi[:30]

[('całodobowo całodobowy', 22.61321701827587),
 ('swłaściwym minister', 22.61321701827587),
 ('głuchowo pogórska', 22.61321701827587),
 ('głuchów głuch', 22.61321701827587),
 ('karczewska wraz', 22.61321701827587),
 ('komorowice oświęcim', 22.61321701827587),
 ('kotowo zielona', 22.61321701827587),
 ('kozienice radom', 22.61321701827587),
 ('mop nie', 22.61321701827587),
 ('mory wola', 22.61321701827587),
 ('mpa łączyć', 22.61321701827587),
 ('polkowice żary', 22.61321701827587),
 ('płońsk olsztyn', 22.61321701827587),
 ('rozwadów strachocina', 22.61321701827587),
 ('skocz komorowice', 22.61321701827587),
 ('skoczów skocz', 22.61321701827587),
 ('tłocznia tłoczny', 22.61321701827587),
 ('tłoczny gaz', 22.61321701827587),
 ('wycinek drzewo', 22.61321701827587),
 ('wycinka wycinek', 22.61321701827587),
 ('dodatnio dodatni', 22.61321701827587),
 ('funkcjonariuszakobiety w', 22.61321701827587),
 ('generalski mianować', 22.61321701827587),
 ('ofiarnia i', 22.61321701827587),
 ('ofiarnie ofi

In [23]:
from collections import Counter
from llr import llr_2x2

In [26]:
llr_dict = {}
for bigram in filtered_bigrams:
    word1, word2 = bigram.split(' ')
    not_word1_word2 = unigram_frequency[word2] - filtered_bigrams[bigram] 
    word1_not_word2 = unigram_frequency[word1] - filtered_bigrams[bigram]
    not_word1_not_word2 = sum(filtered_bigrams.values()) - word1_not_word2 - not_word1_word2 - filtered_bigrams[bigram]
    llr_dict[bigram] = llr_2x2(filtered_bigrams[bigram],not_word1_word2, word1_not_word2, not_word1_not_word2 )

In [27]:
sorted_llr = sorted(llr_dict.items(), key=lambda entry: entry[1] , reverse=True)

In [28]:
sorted_llr[0:30]

[('w wiek', 1512868.9505226118),
 ('o ocean', 672851.4528762214),
 ('ocean ojciec', 672005.4768584454),
 ('lub lubić', 507926.1540169112),
 ('który mowa', 272440.30304606585),
 ('brzmienie brzmieć', 231986.2291352484),
 ('nie on', 218071.38664999907),
 ('od oda', 206144.70497495937),
 ('ojciec który', 187554.41064130655),
 ('mowa w', 177773.8951709359),
 ('określony określić', 158198.62992483506),
 ('może móc', 146529.94176982663),
 ('wiek artykuł', 134558.28035960603),
 ('prawo prawy', 129713.60182514365),
 ('albo alba', 125689.02884591662),
 ('postępowanie postępować', 120219.07562063898),
 ('sprawa sprawić', 118927.15771334711),
 ('otrzymywać brzmienie', 117826.99283330375),
 ('rok numer', 111763.4088798624),
 ('bar bardzo', 101144.60221865875),
 ('bardzo bieżący', 98160.79109145659),
 ('niż niża', 96873.13036181423),
 ('niża nizać', 96873.13036181423),
 ('ten to', 95596.42725932016),
 ('środki środek', 94425.78330453657),
 ('rozporządzenie rozporządzić', 93733.6981369172),
 ('bieżą

In [36]:
sorted_llr[:30]

[('w wiek', 1512868.9505226118),
 ('o ocean', 672851.4528762214),
 ('ocean ojciec', 672005.4768584454),
 ('lub lubić', 507926.1540169112),
 ('który mowa', 272440.30304606585),
 ('brzmienie brzmieć', 231986.2291352484),
 ('nie on', 218071.38664999907),
 ('od oda', 206144.70497495937),
 ('ojciec który', 187554.41064130655),
 ('mowa w', 177773.8951709359),
 ('określony określić', 158198.62992483506),
 ('może móc', 146529.94176982663),
 ('wiek artykuł', 134558.28035960603),
 ('prawo prawy', 129713.60182514365),
 ('albo alba', 125689.02884591662),
 ('postępowanie postępować', 120219.07562063898),
 ('sprawa sprawić', 118927.15771334711),
 ('otrzymywać brzmienie', 117826.99283330375),
 ('rok numer', 111763.4088798624),
 ('bar bardzo', 101144.60221865875),
 ('bardzo bieżący', 98160.79109145659),
 ('niż niża', 96873.13036181423),
 ('niża nizać', 96873.13036181423),
 ('ten to', 95596.42725932016),
 ('środki środek', 94425.78330453657),
 ('rozporządzenie rozporządzić', 93733.6981369172),
 ('bieżą

In [42]:
list(filter(lambda x:x[1]< 30  ,sorted_llr))

[('vina ostatnie', 29.9996449445116),
 ('dania republika', 29.99907760869064),
 ('niderlandy republika', 29.99907760869064),
 ('osad ściekowy', 29.997605890222076),
 ('indywidualny rozkład', 29.997251689466793),
 ('uprawny przed', 29.99593668903981),
 ('zataić bezprawny', 29.99593288399592),
 ('treść polecenie', 29.99587682002675),
 ('przebieg linia', 29.995519822890856),
 ('rezerwowy przeznaczyć', 29.994813740857353),
 ('przeciwnik skarga', 29.994722044228183),
 ('dyscyplinarny polegać', 29.994699481947464),
 ('podatnik dokonać', 29.994343852682505),
 ('wczesno mieć', 29.993120791361434),
 ('socjalna mieszkanie', 29.992278000016086),
 ('wskaźnik charakteryzować', 29.99085549197116),
 ('mój pozycja', 29.9907835269114),
 ('ramy przedpłata', 29.98984542638209),
 ('przekazać dotacja', 29.98916874139104),
 ('dochód rok', 29.985382656974252),
 ('ziemski zastępować', 29.984251514004427),
 ('sprawa zakład', 29.983786597964354),
 ('według wynalazek', 29.98367067588697),
 ('frd odbywać', 29.983

In [29]:
serialize('llr_dict.pickle', llr_dict)

Which measure works better for the problem?

LLR is more suitable than PMI

What would be needed, besides good measure, to build a dictionary of multiword expressions?


Good lemmatization and tokenization 

Can you identify a certain threshold which clearly divides the good expressions from the bad?

In [43]:
30

30